In [35]:
import acquire as a
import prepare as p
import explore as e
import model as m

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [36]:
train, validate, test = p.split_data((p.prepare_data(a.acquire_data())),'churn')

File exists - reading CSV file


In [37]:
len(train.columns)

21

In [38]:
cat_cols = train.columns[train.nunique() < 10].drop('churn').tolist()
num_cols = train.columns[train.nunique() >= 10].drop('customer_id').tolist()

In [39]:
for col in num_cols:
    print(f'{col} vs churn:')
    α = .05
    
    t, p = stats.shapiro(train[col])
    if p < α:
        print('The distributions are not normally distributed.')
    else: 
        print('The distributions are normal.')
    print()

tenure vs churn:
The distributions are not normally distributed.

monthly_charges vs churn:
The distributions are not normally distributed.

total_charges vs churn:
The distributions are not normally distributed.



In [40]:
cat_cols_w_rel = []
cat_cols_wo_rel = []

for col in cat_cols:
    observed = pd.crosstab(train[col],train.churn)
    α = .05
    chi2, p, dof, expected = stats.chi2_contingency(observed)
    
    if p < α:
        cat_cols_w_rel.append(col)
    else:
        cat_cols_wo_rel.append(col)

In [41]:
churned = train[train.churn == 'Yes']
active = train[train.churn == 'No']

quant_cols_w_rel = []
quant_cols_wo_rel = []

for col in num_cols:
    t,p = stats.mannwhitneyu(churned[col], active[col])
    α = .05
    
    if p < α:
        quant_cols_w_rel.append(col)
    else:
        quant_cols_wo_rel.append(col)

In [55]:
ct = pd.crosstab(train[col],train.churn,normalize=0)
ct.shape[0]

2

In [57]:
for col in cat_cols_w_rel:
    print(col)
    ct = pd.crosstab(train[col],train.churn,normalize=0)
    for num in range(ct.shape[0]):
        print(num)
        if num <= (ct.shape[0])-2:
            print((ct.Yes[num]) - (ct.Yes[num+1]))
        else:
            pass
    print()
        
    # ct.

senior_citizen
0
-0.18681072357213305
1

partner
0
0.13666397070387096
1

dependents
0
0.158470544076452
1

multiple_lines
0
-0.021471046297607266
1
-0.026704772563754087
2

online_security
0
0.34114476296809787
1
-0.07421098046098046
2

online_backup
0
0.322552536685299
1
-0.1448298154508901
2

device_protection
0
0.3127215815793402
1
-0.15893287796401984
2

tech_support
0
0.34387333715548685
1
-0.07677668811604718
2

streaming_tv
0
0.26093714783122324
1
-0.22947441106950311
2

streaming_movies
0
0.2613351560719982
1
-0.22904658263774286
2

paperless_billing
0
-0.17633589982381187
1

contract_type
0
0.31841044522408757
1
0.08033107696256135
2

internet_service_type
0
-0.22784015788053727
1
0.3454134058885679
2

payment_type
0
0.015673662788634413
1
-0.29750722927834927
2
0.24007624134171093
3



In [ ]:
sns.countplot(train)